## Create a generative AI runbook to resolve security findings

## Module 1 - Summarize and remediate security findings

```
// Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
// SPDX-License-Identifier: MIT-0
```

### Introduction

In this module, you will complete the following
- Get a finding from Security Hub
- Use Claude 3.7 Sonnet to summarize the finding result
- Use Claude 3.7 Sonnet to generate Python code used for finding discovery
- Use Claude 3.7 Sonnet to generate Python code used for remediating the finding

**NOTE**: You can use SHIFT + ENTER to run each of the cells in this workbook.

**In the first example, get a failed finding from Security Hub for `Security groups should not allow unrestricted access to ports with high risk [EC2.19]`**

<div class="alert alert-block alert-info">
You can safely ignore `Note: you may need to restart the kernel to use updated packages.`
</div>

***

<div style='background-color:#f0f0f0; padding:10px; border-radius:5px;'>
    <h3>Step 1: Get a finding from Security Hub</h3>
    <p>In this step, you will get a failed finding from Security Hub. In this example, we use the AWS SDK for Python (boto3) to access the Security Hub API.</p>
    <p>The Python function will return a JSON representation of the finding. You can also view this finding by visiting Security Hub in the AWS console.</p>
</div>

In [ ]:
%pip install -qU pip boto3 awscli botocore
import boto3

sh = boto3.client('securityhub')

def get_finding(finding_id):
    response = sh.get_findings(
        Filters={
                'ComplianceSecurityControlId': [
                    {
                        'Value': finding_id,
                        'Comparison': 'EQUALS'
                    }
                ],
                'ComplianceStatus':[
                    {
                        'Value': 'FAILED',
                        'Comparison': 'EQUALS'
                    }
                ],
                'ResourceTags': [
                    {
                        'Key': 'Workshop',
                        'Value': 'SampleSecurityGroup',
                        'Comparison': 'EQUALS'
                    }
                ]
            },
        MaxResults=1

    )
    return response['Findings']

sg_finding = get_finding("EC2.19")
security_group_name = sg_finding[0]['Resources'][0]['Details']['AwsEc2SecurityGroup']['GroupName']  
security_group_id = sg_finding[0]['Resources'][0]['Details']['AwsEc2SecurityGroup']['GroupId'] 
print("^ You can safely ignore `Note: you may need to restart the kernel to use updated packages.`\n")
print(sg_finding)
print(f"\nSecurity group name: {security_group_name}")
print(f"Security group ID: {security_group_id}")

<div style='background-color:#f0f0f0; padding:10px; border-radius:5px;'>
    <h3>Step 2: Summarize the finding using Claude 3.7 Sonnet</h3>
    <p>In this step, you will use the boto3 converse API, to call Claude 3.7 Sonnet in Amazon Bedrock.</p>
</div>

In [ ]:
import boto3

modelId = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"

bedrock_client = boto3.client(service_name='bedrock-runtime')
inference_config = {"temperature": 0}

def get_completion(prompt, system_prompt=None):

    # Create the converse method parameters
    converse_api_params = {
        "modelId": modelId,
        "messages": [{"role": "user", "content": [{"text": prompt}]}],
        "inferenceConfig": inference_config,
    }

    # Check if system_prompt is provided and add the system parameter to the converse_api_params dictionary
    if system_prompt:
        converse_api_params["system"] = [{"text": system_prompt}]

    response = bedrock_client.converse(**converse_api_params)

    return response['output']['message']['content'][0]['text']

print("Get completion function defined.")

<div style='background-color:#f0f0f0; padding:10px; border-radius:5px;'>
    <p>After loading the function, you can create your first prompt.</p>
</div>

In [ ]:
system_prompt = "You are an AWS Security Engineer looking to improve the security posture of your organization."

prompt = f"""
    Review the finding and summarize actionable next steps

    <finding>
    {sg_finding}
    </finding>

"""

response = get_completion(prompt, system_prompt)

print(response)

<div style='background-color:#f0f0f0; padding:10px; border-radius:5px;'>
    <h3>Step 3 - Use Claude 3.7 Sonnet to generate Python code used for finding discovery</h3>
    <p>Based on the output in step 2, Claude 3 recommends reviewing the finding and restrict inbound access to only trusted addresses or ranges.</p>
    <p>In this step, you will write a prompt to determine where this security group is being used.</p>
</div>

In [ ]:
# Create a prompt to write a Python function to determine if the security group is being used.

prompt = f"""

Write a simple Python function called is_security_group_in_use() to determine if {security_group_id} security group is being used by an ENI.

"""
print(get_completion(prompt))

<div style='background-color:#f0f0f0; padding:10px; border-radius:5px;'>
    <p>Review the output. Claude 3.7 Sonnet creates a Python function that takes a security group ID, and returns true if the security group is associated to a network interface (ENI).</p>
    <p>Create the function Claude 3.7 Sonnet generated for checking if a security group is being used.</p>
</div>

In [ ]:
# Update this cell with the function for checking if a security group is being used.
# Note: Your function may be different. Generative AI responses are non deterministic. 

import boto3

def is_security_group_in_use(security_group_id='sg-0b38a152debb89064', region='us-east-1'):
    """
    Check if a security group is being used by any ENI.
    
    Args:
        security_group_id (str): The ID of the security group to check
        region (str): AWS region to check in
        
    Returns:
        bool: True if the security group is in use, False otherwise
    """
    # Initialize EC2 client
    ec2 = boto3.client('ec2', region_name=region)
    
    # Describe network interfaces with the specified security group
    response = ec2.describe_network_interfaces(
        Filters=[
            {
                'Name': 'group-id',
                'Values': [security_group_id]
            }
        ]
    )
    
    # If any network interfaces are returned, the security group is in use
    return len(response['NetworkInterfaces']) > 0

# Example usage
if __name__ == "__main__":
    sg_id = security_group_id
    if is_security_group_in_use(sg_id):
        print(f"Security group {sg_id} is in use by at least one ENI")
    else:
        print(f"Security group {sg_id} is not in use by any ENI")


<div style='background-color:#f0f0f0; padding:10px; border-radius:5px;'>
    <p>Claude 3 returns that the security group is not in use.</p>
</div>

<div style='background-color:#f0f0f0; padding:10px; border-radius:5px;'>
    <h3>Step 4: Use Claude 3.7 Sonnet to generate Python code used for remediating the finding</h3>
    <p>In this step, we will ask Claude 3.7 Sonnet to generate a remediation function to update the security group that is not being used and remove port 22.</p>
</div>

<div class="alert alert-block alert-info">
<b>Note:</b> This workshop example is simplified by design. In a production environement, you should follow your change control process and do further diligence before making changes.
</div>

In [ ]:
prompt = f"""Review the finding.

<finding>
{sg_finding}
</finding>

Create a Python function called remove_ssh_access_from_security_group to remove the rule for inbound access on port 22 (SSH) from any IP address.

"""

print(get_completion(prompt))

<div style='background-color:#f0f0f0; padding:10px; border-radius:5px;'>
    <p>Claude 3.7 Sonnet will generate a function to remove port 22 from a security group ID. Carefully review the function. Once you are familiar with how it works, run the function to restrict access to the security group port 22.</p>
</div>

In [ ]:
# Update this cell with the function to remove port 22 from a security group ID
# Note: Your function may be different. Generative AI responses are non deterministic. 

import boto3
from botocore.exceptions import ClientError

def remove_ssh_access_from_security_group(security_group_id, region='us-west-2'):
    """
    Removes inbound SSH access (port 22) from any IP address (0.0.0.0/0) for the specified security group.
    
    Parameters:
    security_group_id (str): The ID of the security group to modify
    region (str): AWS region where the security group is located
    
    Returns:
    dict: Response from the revoke_security_group_ingress API call
    """
    try:
        # Create EC2 client
        ec2_client = boto3.client('ec2', region_name=region)
        
        # Revoke the SSH ingress rule
        response = ec2_client.revoke_security_group_ingress(
            GroupId=security_group_id,
            IpPermissions=[
                {
                    'IpProtocol': 'tcp',
                    'FromPort': 22,
                    'ToPort': 22,
                    'IpRanges': [
                        {
                            'CidrIp': '0.0.0.0/0'
                        }
                    ]
                }
            ]
        )
        
        print(f"Successfully removed SSH access from security group {security_group_id}")
        return response
        
    except ClientError as e:
        print(f"Error removing SSH access: {e}")
        raise

# Example usage:
remove_ssh_access_from_security_group(security_group_id)

<div style='background-color:#f0f0f0; padding:10px; border-radius:5px;'>
    <p>The rule will be removed from the security group. You can verify by checking in the AWS console.</p>
</div>